# Setup

In [ ]:
%matplotlib inline

In [ ]:
from utils_med import *

In [ ]:
#%load utils_med.py

# Innledning

I denne notebooken bruker vi deep learning på medisinske bilder. Vi skal trene et system som kan stille diagnose basert på røntgenbilder

Vanlige røntgenbilder er den mest brukte metoden for å stille diagnose fra medisinske bilder. Fra [Wilhelm Conrad Röntgen](https://en.wikipedia.org/wiki/Wilhelm_R%C3%B6ntgen) oppdaget at røntgenstråler kan brukes til å identifisere bein i kroppen i 1895 og frem til 2010 har det blitt tatt **5 milliarder røntgenbilder**. 

<img width=40% src="assets/X-Ray.png">

I klinikken ser radiologer eller radiografer på bildene for å kunne stille en diagnose. Siden det tas så mange røntgenbilder, er dette en tidkrevende jobb.

I denne workshopen skal du trene et nevralt nettverk som kan se om et røntgenbilde av bryst viser tegn til lungebetennelse eller ikke. Det vil si, klassifisere røntgenbilder med enten «No Finding» eller «Pneumonia». 

Dette er en viktig oppgave: Om en kjapt og presist kan utelukke en diagnose så slipper pasienten videre undersøkelser (og bekymringer) i den retningen, og en kan fokusere på andre potensielle diagnoser. En slik kunstig intelligens, også kalt et AI-system, kan potensielt være tidsbesparende. Radiologen kan bruke tiden på de vanskeligere tilfellene.

Som du skal se, AI-systemet du setter opp er i stand til å stille diagnose på nivå med eksperter!

## Datamateriale

<a href="https://www.nih.gov/news-events/news-releases/nih-clinical-center-provides-one-largest-publicly-available-chest-x-ray-datasets-scientific-community">Treningsdata</a> vi bruker er fra National Institute of Health i USA, og ligger åpent tilgjengelig på nettet: https://www.kaggle.com/nih-chest-xrays

Hvert bilde er annotert med en eller flere av <a href="https://www.kaggle.com/nih-chest-xrays/data">14 ulike diagnoser</a>, blant annet Pneumonia (lungebetennelse).

I denne notebooken ser vi kun på klassifisering av lungebetennelse versus resten. «Pneumonia» versus «No Pneumonia». 

### Tilrettelegging av data

In [ ]:
NB_DIR = %pwd
DATA_DIR = f'{NB_DIR}/data/XRay/'

Jeg har organisert dataene i kataloger etter diagnose. Treningsdata og valideringsdata er delt opp i ulike underkataloger. 

In [ ]:
!tree -L 2 $DATA_DIR

In [ ]:
train_dir = f'{DATA_DIR}train/'
val_dir = f'{DATA_DIR}val/'

Inne i hver katalog ligger bildene

In [ ]:
!tree -L 3 $DATA_DIR

Vi samler treningsbildene og valideringsbildene i to lister:

In [ ]:
from glob import glob

In [ ]:
train_images = glob(f'{train_dir}/*/*.png')
val_images = glob(f'{val_dir}/*/*.png')

In [ ]:
train_images[:5]

In [ ]:
len(train_images), len(val_images)

In [ ]:
import matplotlib.pyplot as plt

img = cv2.imread(train_images[0])

plt.imshow(img)
plt.axis('Off')
plt.show()

# Nevralt nettverk

Vi kan nøyaktig de samme nettverkene som for Cifar-10 i forrige del av workshopen. Men denne gang mater vi inn røntgenbilder og ber om å få ut en av to ulike diagnoser: enten «no finding» eller «pneumonia», markert som 0 eller 1.

## Enkelt (moderne) nettverk

In [ ]:
# Nedskalerer bildene til 50x50
img_width, img_height = 50, 50

In [ ]:
nb_train_samples = len(train_images)
nb_validation_samples = len(val_images)
batch_size = 16

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(img_width, img_height, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=2,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Som du ser så trenes nettverket. Men resultatene er så som så... Husk at det bare er to mulige utfall. Vill gjetning gir 50% accuracy på treningsdata.

Her er fem mulige årsaker til lav accuracy

1. Trent for kort. Fiks: tren mer (forsøk gjerne dette)
2. For små bilder. Vi nedskalerte til 50x50. Fiks: velg noe større (koster i form av tid og minne på GPU)
3. For lite data. Fiksbart siden vi har 112.000 bilder til rådighet (men ikke fiksbart her i dag, siden vi kun leker med et lite utvalg)
4. For simpel nettverksarkitektur. 
5. For simpelt treningsregime. Fiks: mer forsiktig valg av ting som learning rate (gjerne varierende lr under trening, batch_size, hvilken optimerer, osv)

**Vi undersøker punkt 4**

## DenseNet121

Her laster vi inn DenseNet121-nettverket vi brukte på Cifar-10 i `1.0-neural_networks.ipynb`:

In [ ]:
model = densenet121_xray_pretrained()

In [ ]:
model.compile(optimizer=SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=1)

Dette fungerer ikke bra... 

Vi kunne trent mer (du må gjerne forsøke), men det vil være lite forbedring før vi putter inn mer data. 

Det er fordi pneumonia er et subtilt fenomen på røntgen. Det er små nyanser i mønstrene i bildet som avgjør hvorvidt pneumonia settes som diagnose. Jo mindre nyanser, jo mer treningsdata kreves. 

*Spurv med kanoner, men nesten ingen spurv i sikte.*

Hva hvis vi hadde matet inn alle 112.000 bildene i (nøyaktig) dette nettverket? 

Jo, et fantastisk godt resultat! 

https://stanfordmlgroup.github.io/projects/chexnet/

<img src="assets/chexnet-1.png">

<img src="assets/chexnet-2.png">

Dette illustrerer viktigheten av treningsdata. Metoder som er uhyre gode kan fungere dårlig, helt til en plutselig gir dem nok treningsdata. 

    Gode metoder + tilstrekkelig med data + tilstrekkelig med regnekraft = suksess

alle tre må til. 

Snakk med Sathiesh om du vil vite mer om hvordan du kan få gode resultater på disse dataene. Han har reprodusert resultatatet fra Stanford, og arbeider også med en utvidelse til andre typer diagnoser på andre typer røntgenbilder. 

<img width=40% src="assets/sathiesh.jpg">